In [1]:
import sys; sys.path.append('../..')

In [2]:
import os
import pickle

from IPython.display import display, Markdown
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

from attre2vec.evaluation import metrics as ae_mtr

In [3]:
def evaluate_edge_clustering(model_names, dataset_names, base_path):
    res = {}
    
    for dataset_name in tqdm(dataset_names, desc='Dataset'):
        dataset_path = os.path.join(base_path, f'data/datasets/{dataset_name}.pkl')

        with open(dataset_path, 'rb') as fin:
            ds = pickle.load(fin)


        for mn in tqdm(model_names, desc='Models', leave=False):
            accs = []
            for ds_idx in tqdm(range(ds['num_datasets']), desc='Dataset samples', leave=False):
                y = np.array([
                    *ds['Xy'][ds_idx]['train']['y'],
                    *ds['Xy'][ds_idx]['val']['y'],
                    *ds['Xy'][ds_idx]['test']['y'],
                ])

                vp = os.path.join(
                    base_path, 'data/vectors', dataset_name, mn, f'{ds_idx}.pkl'
                )
                with open(vp, 'rb') as fin:
                    embs = np.array(pickle.load(fin))

                accs.append(ae_mtr.acc(embs=embs, y_true=y))

            mean = np.round(np.mean(accs) * 100.0, 2)
            std = np.round(np.std(accs) * 100.0, 2)

            res[(dataset_name, mn)] = {'mean': mean, 'std': std}
    
    return res


In [4]:
datasets = ['cora', 'citeseer', 'pubmed']
    
model_names = [
    'BL_dw/nf/full', 'BL_dw/nfef/full',
    'BL_n2v/nf/full', 'BL_n2v/nfef/full',
    'BL_sdne/nf/full', 'BL_sdne/nfef/full',
    'BL_struc2vec/nf/full', 'BL_struc2vec/nfef/full',
    'BL_graphsage/nf/full', 'BL_graphsage/nfef/full',
    
    'BL_simple/full',
    
    'BL_line2vec',
    
    'AttrE2vec_Avg', 'AttrE2vec_Exp', 'AttrE2vec_GRU', 'AttrE2vec_ConcatGRU',
    
    'MLP_dw/MLP2', 'MLP_n2v/MLP2', 'MLP_sdne/MLP2', 'MLP_struc2vec/MLP2', 'MLP_graphsage/MLP2', 
    'MLP_dw/MLP3', 'MLP_n2v/MLP3', 'MLP_sdne/MLP3', 'MLP_struc2vec/MLP3', 'MLP_graphsage/MLP3', 
]

In [5]:
res = evaluate_edge_clustering(
    model_names=model_names, 
    dataset_names=datasets,
    base_path='../../'
)

In [6]:
df = pd.DataFrame.from_records(
    [(d, m, f'{v["mean"]} +/- {v["std"]}') for (d, m), v in res.items()],
    columns=['dataset', 'model', 'accuracy'],
)
     
     
df = df.pivot(index='model', columns='dataset')
df

accuracy                                
dataset                       citeseer            cora          pubmed
model                                                                 
AttrE2vec_Avg            59.82 +/- 3.3  65.42 +/- 1.71  48.86 +/- 2.46
AttrE2vec_ConcatGRU     60.71 +/- 2.75   66.0 +/- 2.21  50.27 +/- 3.75
AttrE2vec_Exp           59.07 +/- 4.65  66.36 +/- 3.62  48.02 +/- 2.55
AttrE2vec_GRU           60.16 +/- 2.25  66.15 +/- 3.71  49.41 +/- 1.49
BL_dw/nf/full           28.89 +/- 1.06  21.93 +/- 0.86   27.24 +/- 0.5
BL_dw/nfef/full         54.13 +/- 2.73   54.7 +/- 5.85  46.33 +/- 1.53
BL_graphsage/nf/full    18.79 +/- 0.62   17.7 +/- 1.05  27.04 +/- 0.71
BL_graphsage/nfef/full  54.06 +/- 2.54  54.82 +/- 6.86  46.49 +/- 1.64
BL_line2vec             54.73 +/- 2.56   63.5 +/- 1.92  55.26 +/- 1.36
BL_n2v/nf/full          26.82 +/- 0.67  21.32 +/- 0.62  27.17 +/- 0.74
BL_n2v/nfef/full        54.13 +/- 2.73   54.7 +/- 5.85  46.33 +/- 1.53
BL_sdne/nf/full          21.01 +/- 0.5  17.97 +/- 0.47  31.38 +/- 0.69
BL_sdne/nfef/full       55.29 +/- 2.06  55.43 +/- 4.63  46.33 +/- 1.53
BL_simple/full          54.13 +/- 2.73  54.64 +/- 5.86  46.33 +/- 1.53
BL_struc2vec/nf/full    25.21 +/- 1.33  20.15 +/- 0.64  32.02 +/- 1.49
BL_struc2vec/nfef/full  55.59 +/- 1.51  52.47 +/- 6.52  46.32 +/- 1.29
MLP_dw/MLP2             26.36 +/- 1.37  21.06 +/- 0.57   27.4 +/- 0.93
MLP_dw/MLP3             48.74 +/- 4.03  47.38 +/- 4.72   46.49 +/- 1.2
MLP_graphsage/MLP2      18.92 +/- 0.98  17.89 +/- 0.85  27.09 +/- 0.81
MLP_graphsage/MLP3      48.79 +/- 4.04  47.49 +/- 5.41  45.15 +/- 1.54
MLP_n2v/MLP2            26.37 +/- 1.64  21.31 +/- 0.98  27.67 +/- 0.78
MLP_n2v/MLP3              50.8 +/- 2.3  48.48 +/- 3.38  46.15 +/- 1.43
MLP_sdne/MLP2           22.27 +/- 0.76  17.15 +/- 0.36  28.44 +/- 1.21
MLP_sdne/MLP3           46.17 +/- 3.15  44.87 +/- 3.54  45.74 +/- 1.89
MLP_struc2vec/MLP2      24.22 +/- 0.83  19.56 +/- 0.49   31.31 +/- 1.7
MLP_struc2vec/MLP3      47.35 +/- 3.73  44.38 +/- 3.04   45.4 +/- 1.72

In [7]:
with open('../../data/paper/table-edge-clustering-acc.tex', 'w') as fout:
    fout.write(df.to_latex())